In [1]:
from pystac_client import Client

from mccn.load import csv_load

In [2]:
catalog = Client.open("http://115.146.84.224:8082")

In [4]:
collection = catalog.get_collection(collection_id="point_data")

In [7]:
items = list(collection.get_all_items())

In [8]:
grandview = items[2]

In [11]:
ds = csv_load(grandview)

In [ ]:
ds